# Intro

#### This script was written to perform an experimental test on a robot (Ambrogio 🤖❤️) we built for the Cajal Advanced Neuroscience Training Porgramme. A face recognition function that plays through a web-streaming camera gives the input to Ambrogio to turn his back and run away whenever he meets human faces! The first part of the task (camera stream and face recognition) is written in python. Then, a serial port comunication is established, where an Arduino runs a .ino file to complete the second part of the task (moving the wheels to run away:). The audio connection through the ears of the robot is also initialized in this script, but it doesn't generate any output for the current task.

##### The script was modified and implemented from several scripts in the following github repositories and open web resources:  
https://github.com/NoBlackBoxes/LastBlackBox/tree/master/course , https://github.com/NoBlackBoxes/LastBlackBox/tree/master/course/bootcamp/day_5/resources ,
https://picamera.readthedocs.io/en/release-1.13/recipes2.html

!/usr/bin/python3 #Written in python3

- Run this script, then point a web browser at "http://<this-IP-address>:8000/index.html" (for example  http://192.168.137.183:8000/index.html) 

- Note: needs simplejpeg to be installed (pip3 install simplejpeg).


In [19]:
pip install simplejpeg

Hey! I'm editing the standard library!
Note: you may need to restart the kernel to use updated packages.


In [20]:
#Import needed libraries

## FOR LIVE STREAM
import io
import logging
import socketserver
from http import server
from threading import Condition

## FOR IMAGE AND AUDIO PROCESSING
import cv2
import numpy as np

import socket
import struct
import pyaudio

import serial
import time

## FOR HARDWARE CAMERA
from picamera2 import Picamera2   
from picamera2.encoders import JpegEncoder
from picamera2.outputs import FileOutput

PAGE = """\
<html>
<head>
<head>
<head>
<head>

<title>picamera2 MJPEG streaming demo</title>
</head>
<body>
<h1>Picamera2 MJPEG Streaming Demo</h1>
<img src="stream.mjpg" width="640" height="480" />
</body>
</html>
"""

ModuleNotFoundError: No module named 'cv2'

In [9]:
# Configure serial port.
## I first create a 'ser' object
### I then set the rate at which data are trasmitted over the serial connection (baudrate) based on my device (Arduino and RasperryPi)
#### I lastly set the port of the object 'ser' to the physical serial port that connect the Arduino and the RaspPi

ser = serial.Serial()
ser.baudrate = 19200
ser.port = '/dev/ttyUSB0'

NameError: name 'serial' is not defined

In [10]:
# Open serial port to send/receive data

ser.open()
time.sleep(2.00) # Wait for connection before sending any data, for the sake of synchronization

NameError: name 'ser' is not defined

In [11]:
# Audio connection
## Not useful for the current task, but still Ambrogio has good ears that we need to initialize for further task:)

##Setting up the audio parameter for audio processing using the PyAudio library 

CHUNK_SIZE = 4096           # Buffer size in bytes (n of audio frame processed at time)
FORMAT = pyaudio.paInt16    # Data type
CHANNELS = 2                # Number of channels, L and R ear
RATE = 22050                # Sample rate (Hz)

NameError: name 'pyaudio' is not defined

In [ ]:
# Initialize PyAudio
## Creating the 'audio' object as the interface to my robot's audio hardware
audio = pyaudio.PyAudio()

In [ ]:
# Open the audio stream, in order to then use the 'stream' object to read or perform real-time audio processing
stream = audio.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK_SIZE)

In [ ]:
# WEB STREAMING CAMERA
##The classes defined are inherited from IO (used for streaming video over HTTP) and server.BaseHTTPRequestHandler (used to handle HTTP requests)

class StreamingOutput(io.BufferedIOBase):
    def __init__(self):
        self.frame = None #initializing the attribute 'frame' used to store the latest video frame as a bytes-like object
        self.condition = Condition() #initializing 'condition' for synchronization -> wait for a condition before proceeding

    def write(self, buf):
        with self.condition:
            self.frame = buf
            self.condition.notify_all()

class StreamingHandler(server.BaseHTTPRequestHandler):
    def do_GET(self):  #for GET requests 
        if self.path == '/':
            self.send_response(301)
            self.send_header('Location', '/index.html')
            self.end_headers()
        elif self.path == '/index.html':
            content = PAGE.encode('utf-8')
            self.send_response(200)
            self.send_header('Content-Type', 'text/html')
            self.send_header('Content-Length', len(content))
            self.end_headers()
            self.wfile.write(content)
        elif self.path == '/stream.mjpg': 
            self.send_response(200)
            self.send_header('Age', 0)
            self.send_header('Cache-Control', 'no-cache, private')
            self.send_header('Pragma', 'no-cache')
            self.send_header('Content-Type', 'multipart/x-mixed-replace; boundary=FRAME')
            self.end_headers()
            
            #Put the code in a "try" block to make it run indefinitely, continously serving video frames
            try:
                while True:
                    with output.condition:
                        output.condition.wait() #it waits for a condition to be notified indicating that a new frame is ready
                        frame = output.frame #it retrieves the latest frame 

                        # Open the cv2 library to decode the image from a binary buffer
                        ## The image is encoded in bytes, needs to be converted to a numpy array
                        ### Color mode of the image is also specified
                        #### 'Frame' variable results in the decoded image 
                        
                        frame = cv2.imdecode(np.frombuffer(frame, dtype=np.uint8),
                                             cv2.IMREAD_COLOR) 
                        
                        
                       
                        # HERE CAN GO ALL IMAGE PROCESSING
                
                        frame = frame[::-1].copy() #flip image horizontally + create a copy of the modified frame
                        det = cv2.CascadeClassifier("haarcascade_frontalface_default.xml") #load a Haar Cascade classifier for frontal face detection (pre-trained model)

                        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert the frame to greyscale, the face-det algorithm works on greyscale

                        #detect face and adjust parameters for face detection sensitivy and accuracy
                        rects = det.detectMultiScale(gray, 
                                                     scaleFactor=1.1, 
                                                     minNeighbors=5, 
                                                     minSize=(30, 30), 
                                                     flags=cv2.CASCADE_SCALE_IMAGE)

                        for (x, y, w, h) in rects:
                            # draw a rectangle around the detected face where:
                            ## x: x location
                            ## y: y location
                            ## w: width of the rectangle 
                            ## h: height of the rectangle
                            ## Remember, order in images: [y, x, channel]
                            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 20)
                            
                            # Face detected. Send command to robot to run!
                            #In order to make the robot run we are sending data to the serial port to talk with the Arduino, which will run the following .ino script to make the wheels move "https://github.com/NoBlackBoxes/LastBlackBox/blob/master/course/bootcamp/day_2/resources/arduino/servo_test/servo_test.ino"
                            ser.write(b'b')
                            
        

                        cv2.imwrite("test_face.jpg", frame) #save the frame with rectangle around the detected face
                            
                        # and now convert it back to JPEG to stream it
                    _, frame = cv2.imencode('.JPEG', frame)

                    self.wfile.write(b'--FRAME\r\n')
                    self.send_header('Content-Type', 'image/jpeg')
                    self.send_header('Content-Length', len(frame))
                    self.end_headers()
                    self.wfile.write(frame)
                    self.wfile.write(b'\r\n')
                    
                    #AUDIO RECORDING, NOT USED FOR THE CURRENT TASK
                    
                    # Read audio data from the stream
                    raw_data = stream.read(CHUNK_SIZE)

                    # Convert raw_data to left and right channel
                    interleaved_data = np.frombuffer(raw_data, dtype=np.int16)
                    left = interleaved_data[::2]
                    right = interleaved_data[1::2]

                    # Report volume (on left)
                    print("L: {0:.2f}, R: {1:.2f}".format(np.mean(np.abs(left)), np.mean(np.abs(right))))
                    value = float(np.mean(np.abs(left)))
                    
                        
                    
            except Exception as e:
                logging.warning(
                    'Removed streaming client %s: %s',
                    self.client_address, str(e))
        else:
            self.send_error(404)
            self.end_headers()


class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True

In [ ]:
class StreamingServer(socketserver.ThreadingMixIn, server.HTTPServer):
    allow_reuse_address = True
    daemon_threads = True

# Open the camera and stream a low res image
picam2 = Picamera2()
picam2.configure(picam2.create_video_configuration(main={"size": (640, 480)}))
output = StreamingOutput()
picam2.start_recording(JpegEncoder(), FileOutput(output))

try:
    address = ('', 8000)
    server = StreamingServer(address, StreamingHandler)
    server.serve_forever()
finally:
    picam2.stop_recording()
    ser.close()